# Save Serializer Data to the Database

## Basic Usage

In [1]:
from datetime import date

from django.test import RequestFactory

from organizer.models import Tag
from organizer.serializers import StartupSerializer, TagSerializer

In [2]:
fake_request = RequestFactory().get("/api/v1/tag/")

In [3]:
data = {"name": "new tag"}
s_tag = TagSerializer(
    data=data,
    # request is necessary for full URL reversal
    # needed by url field of HyperlinkedModelSerializer
    context={"request": fake_request},
)

In [4]:
s_tag.is_valid()

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.

In [ ]:
s_tag.errors

In [ ]:
Tag.objects.filter(name="new tag").exists()

In [ ]:
s_tag.save()

In [ ]:
Tag.objects.filter(name="new tag").exists()

## Errors and Extra Values

In [ ]:
s_tag = TagSerializer(
    data={},
    context={"request": fake_request},
)

In [ ]:
s_tag.is_valid()

In [ ]:
s_tag.errors

In [ ]:
s_tag = TagSerializer(
    data={
        "name": 'newer tag',
        "ignored": "this value is ignored"
    },
    context={"request": fake_request},
)

In [ ]:
s_tag.is_valid()

In [ ]:
s_tag.errors

In [ ]:
s_tag.save()

## Nested Serializer Handling

In [ ]:
s_startup = StartupSerializer(
    data=dict(
        name="JamBon Software LLC",
        slug="jambon-software-llc",
        description="Consulting & training for web and mobile products.",
        founded_date=date(2013, 1, 13),
        contact="django@jambonsw.com",  # not a real email
        website="https://jambonsw.com",
        tags=[
            {"name": "newest tag"},
        ],
    ),
    context={"request": RequestFactory().get("/api/v1/startup/")},
)

In [ ]:
s_startup.is_valid()

In [ ]:
s_startup.errors

In [ ]:
# serializers inside serializers is not supported
# we will therefore need to modify all our serializers
# that use other serializers for model relations:
# StartupSerializer, NewsLinkSerializer, PostSerializer
try:
    s_startup.save()
except AssertionError as err:
    print(err)

## Notebook Cleanup

In [ ]:
Tag.objects.filter(name__endswith=" tag").delete()